# Shadow Tomography by means of Classical Shadows
Below follows the procedure outlined in the paper *Predicting many properties of a Quantum System from Very Few Measurments* 

## Construction of the Classical Shadow

Module Imports

In [1]:
import numpy as np
import qutip

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import *
from qiskit.primitives import StatevectorSampler

First with a single qubit

In [47]:
# A set of unitary evolutions (the one defined below is tomographically complete for a system of single qubit).

# Pauli Matrices including the identity.
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

unitary_ensemble = [I, X, Y, Z]

# Randomly select a unity from the ensemble.
i = np.random.randint(0,len(unitary_ensemble))
unitary = unitary_ensemble[i]

# -------------------------------------------------------------------------------------------------------------

# Rotate the state and measure in the computational basis.

# Instantiate quantum and classical registers each of 1 bit.
qr = QuantumRegister (1, 'q')
cr = ClassicalRegister(1, 'measurment')

# Instantiate a quantum circuit of 1 qubit and 1 cbit.
qc = QuantumCircuit(qr, cr)
qc.append (
    HGate(),
    [0]
)
qc.append (                                         # Add a new gate to the circuit.
    UnitaryGate(unitary, "My Gate"),                # Gate to be added.
    [0]                                             # Qubit(s) on which it is acting.
)
qc.measure(0,0)

sampler = StatevectorSampler(default_shots = 1)     # Instantiate a sampler that runs the cicuit once.
result = sampler.run([qc]).result()[0]              # Run the sampler on the quantum circuit and store the result.

counts = result.data.measurment.get_counts()        # Store the counts for number of 0's and 1's measured in the computational basis.

# Get the measured basis state.
if '0' in counts.keys():
    b = 0
else:
    b = 1   

# -------------------------------------------------------------------------------------------------------------                                  

# Apply the inverse of the unitary to the computational basis state obtained from the measurement.

if b == 0:
    b_state = np.array(([1,0],[0,0]))
else:
    b_state = np.array(([0,0],[0,1]))


print(np.matmul(np.matmul(np.matrix(unitary).H, b_state), unitary))


[[1 0]
 [0 0]]


## Median of Means Prediction
Based on the psudo code in *Predicting Many Properties of a Quantum System from Very Few Measurments*

In [39]:
# Indexing is wrong for a lot of the loops. Paper starts indexing from 1 but python indexes from 0.

def LinearPredictions(observables, shadow, K):
    
    M = len(observables) # Number of observables to predict.
    N = len(shadow) # Size of classical shadow.

    size = np.floor_divide(N, K)
    estimators = []
    for k in range(1, K+1):

        rho = np.zeros(shadow[0].shape)

        for i in range(((k-1)*size + 1), (k*size + 1)):
            rho += shadow[i]

        rho *= (1/size)

        estimators.append(rho)

    for i in range(0, M):

        means = []
        for k in range(0, N):
            means.append(np.matmul(observables[i], estimators[k]))

        expectation = np.median(means)
        print("The expectation value of observable ", i, " on the prepared state is:", expectation)
